## Semantic Segmentation

So far we have seen image classification problem where we determine the class of an image. But in semantic segmentation we will determine the category of each pixel of an image. For example, a cat is running on the field. Here we will determine cat, grass, background etc. for that image.

In semantic segmentation we just label each pixel of the image but don't differentiate instances. For example, if two cow is next to each other in an image it will only label as cow but will not differntiate as two different cow. This is the shortcomings of semantic segmentation.

![alt text](images/ss1.PNG)

 ## Semantic Segmentation Idea

 ##### Idea 1 (Sliding Window)

 We can do semantic segmentation using CNN. The idea is to apply sliding window approach. We take our input image and break it into many many small crops of this image. Now we will consider each of this crop as an image and consider as a classification problem. We will classify the central pixel of each crop. This would work to some extent. But this is computationally bad. Because this would be computationally expensive to run forward and backward pass. However, we can share features between two patches that are right next to each other. Because the convolutional features of those patches go through same Convolutional layers. Therfore, we can share lot of computation when applying this to separate passes. However, this is an terrible idea.

![alt text](images/ss2.PNG)

##### Idea 2 (Convolution)
"The idea of fully connected layer". In this process instead of extracting individual patches of the image we will take the whole image as input an pass it through a whole giant stack of convolutional layers with no fully connected layers or anything. By doing so we will end up getting a output tensors of size [C X H X W]. Here, C is the number of categories, H and W are height and width respectively. This tensor consist of just the classification scores for evey pixels of the input image. We will compute clssification loss(cross entropy loss), backpropagation for each pixel. Well the training data should be labeled with each pixel for this task which is very expensive.

**Problems:**
- Classification architecture reduces the spatial size of the image when going deeper, but the spatial size remains the same in semantic segmentation.
- convolution at original image resolution at every step is very expensive.


![alt text](images/ss3.PNG)

**Solution**:
Rather than keeping the spatial size of the image same throughtout all the convolutional layers we can design the architecture like something else. In this architecure we will run some convolutional layer with the same spatial size, then downsapmple that feature map using maxpooling or striped convolutions. Then rather than trnasitioning this downsampled image to a fully connected layer we will again increase the spatial size of the image so that the image can have the same size as the input.

![alt text](images/ss4.PNG/)


So far we have seen, downsampling layer in CNNs, but nevers discussed upsampling.
The idea for upsampling is something like unpooling the. Several techniques for up-sampling.
- Nearest Neighbor Unpooling: Here in our example, the input is 2X2 image and after upsampling the image is 4X4. Here we have done a two by two stride to the neares neighbor unpooling where we have duplicated that element for every point in our two by two receptive field of the unpooling region.

- Bed of nails: In this case for the same input(2X2 receptive field), we will put all zeros except for the element of the unpooling region. In this case we have put the inputs in the upper left corner and everything else is zero.

![alt text](images/ss5.PNG)

- **Max unpooling**: In this process we will remember the position of the value in the image which is used and put in the downsampled image using max pooling. Now when we unpooling the downsampled image, we will put the value in that position, and every other position is filled with zeros.

By doing so, it becomes more of a sense to keep the spatial information same from the original image. Because when we put the value in the same slot it might handle the fine detatils of the image.

![alt text](images/ss6.PNG)

- **Traspose Convolution**: The above discussed methods are fixed function. They don't learn how to upsample. There is a convolution type name "striped convolution" which is a learnable layer that can learn the way the network wants to perform down sampling. By the analogy with that there is a type of layer called "Transpose convolution" that can learn while upsampling. So it upsamples the feature map and lean some weights about how it wants to do that upsampling as well.

    But stripped downsampling is actually a convolution. How? Well, remember that, when we have a input image of size 4x4, with filter size 3, padding 1, stride 1 we will have output of size 4x4. But if we use stride 2 then the output will be of size 2x2. This means image is downsampled with some learnable weights. Stride refers to the ratio of how much we move the kernal in the input and in the output. Therefore in this case the stride is 2.

    The upsampling is therefore nothing but transpose convolution. But here the input is 2x2 and output will be 4x4. Here we don't use dot product between the kernal and the weithts. Instead we will take one value from the input and multiply it all the weights of the kernal and put all the values having shape of kernal size in the output. Here the stride is 1/2. Therefore sometimes it is called fractional convolution. The tricky part is sometimes the receptive fields in the ouput might overlap. we will add those values. So we can say, this is kind of learnable upsampling because we used learned convolutional filter weights.

![alt text](images/ss7.PNG) ![alt text](images/ss8.PNG)

![alt text](images/ss9.PNG) 
![alt text](images/ss9.PNG) 

![alt text](images/ss12.PNG)

Therefore we can take an input image and pass it through this gigantic convolutional layer consisting of downsampling and upsampling for semantic Segmentation. We can train this whole thing with backpropagation using cross entropy loss over every pixel.